In [2]:
#라이브러리 불러오기
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 애플코딕 설치되어 있을때
plt.rcParams['font.family'] = 'AppleGothic'

---
## 전기차 충전소 현황
- https://www.data.go.kr/data/15121455/fileData.do#tab-layer-file


In [3]:
#원본 가져오기
charger_df = pd.DataFrame(pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20231031.csv', encoding='cp949'))
charger_df = charger_df[charger_df['시도'] == '서울특별시']
#데이터 확인
charger_df.head(3) 
charger_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45381 entries, 0 to 273269
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   설치년도     45381 non-null  int64 
 1   시도       45381 non-null  object
 2   군구       45300 non-null  object
 3   주소       45381 non-null  object
 4   충전소명     45381 non-null  object
 5   시설구분(대)  45338 non-null  object
 6   시설구분(소)  45381 non-null  object
 7   기종(대)    45381 non-null  object
 8   기종(소)    45381 non-null  object
 9   운영기관(대)  45381 non-null  object
 10  운영기관(소)  45381 non-null  object
 11  급속충전량    348 non-null    object
 12  충전기타입    45381 non-null  object
 13  이용자제한    45381 non-null  object
 14  충전기ID    45381 non-null  int64 
 15  충전소ID    45381 non-null  object
 16  위도경도     44644 non-null  object
dtypes: int64(2), object(15)
memory usage: 6.2+ MB


/var/folders/tz/h0d7hy211kj12my58s3b3m7m0000gn/T/ipykernel_7066/3748189127.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  charger_df = pd.DataFrame(pd.read_csv('/Users/sookyeong/Documents/내배캠/팀프로젝트/project5_final/한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20231031.csv', encoding='cp949'))


- 전처리 1 : 좌표
   - 좌표 null 구하기
   - 카카오 API : null 25개
   - geocorder : null 331개
   - geocorder + 카카오 API : null 16개 (채택) -> null 16개 삭제

In [4]:
# # 카카오 API
# import pandas as pd
# import requests

# KAKAO_REST_API_KEY = '13ee922882dc101cbce7c7a616661d77'  # 발급받은 API KEY

# def convert_address_to_coordinates(address):
#     """
#     입력받은 주소를 WGS84 좌표계 좌표로 변환
#     """
#     url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
#     headers = {'Authorization': f'KakaoAK {KAKAO_REST_API_KEY}'}
 
#     response = requests.get(url, headers=headers)
 
#     if response.status_code == 200:
#         data = response.json()
#         if "documents" in data and data["documents"]:
#             longitude = float(data["documents"][0]["x"])
#             latitude = float(data["documents"][0]["y"])
#             return latitude, longitude
#         else:
#             print(f"No coordinates found for address: {address}")
#     else:
#         print(f"Error: Status code {response.status_code}")
    
#     return None, None

# # 데이터프레임의 각 행에 대해 처리
# for index, row in charger_df.iterrows():
#     if pd.isnull(row['위도경도']):
#         latitude, longitude = convert_address_to_coordinates(row['주소'])
#         if latitude is not None and longitude is not None:
#             charger_df.at[index, '위도'] = latitude
#             charger_df.at[index, '경도'] = longitude
#     else:
#         latitude, longitude = map(float, row['위도경도'].split(','))
#         charger_df.at[index, '위도'] = latitude
#         charger_df.at[index, '경도'] = longitude

In [5]:
charger_df.isna().sum()
# --> 카카오API시 null값 25개

설치년도           0
시도             0
군구            81
주소             0
충전소명           0
시설구분(대)       43
시설구분(소)        0
기종(대)          0
기종(소)          0
운영기관(대)        0
운영기관(소)        0
급속충전량      45033
충전기타입          0
이용자제한          0
충전기ID          0
충전소ID          0
위도경도         737
dtype: int64

In [6]:
from geopy.geocoders import Nominatim

# geopy의 Nominatim 객체 생성
geolocator = Nominatim(user_agent="my_geocoder")

def get_point(df):
    if pd.isnull(df['위도경도']):
        #주소의 괄호 앞부분만 사용, 모두 사용하면 null 512개 나옴
        address = df['주소'].split('(')[0]
        try:
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
        except:
            pass
    else:
        latitude, longitude = df['위도경도'].split(',')
        return float(latitude), float(longitude)

# apply 함수를 사용하여 위도와 경도를 추출하고 새로운 열에 추가
charger_df[['위도', '경도']] = charger_df.apply(get_point, axis=1, result_type='expand')


In [7]:
charger_df.isna().sum()

설치년도           0
시도             0
군구            81
주소             0
충전소명           0
시설구분(대)       43
시설구분(소)        0
기종(대)          0
기종(소)          0
운영기관(대)        0
운영기관(소)        0
급속충전량      45033
충전기타입          0
이용자제한          0
충전기ID          0
충전소ID          0
위도경도         737
위도           331
경도           331
dtype: int64

In [8]:
# 위도, 경도 null 331개 줄이기 위해서 카카오 api로 한번 더 좌표 호출
import pandas as pd
import requests

KAKAO_REST_API_KEY = '13ee922882dc101cbce7c7a616661d77'  # 발급받은 API KEY

def convert_address_to_coordinates(address):
    """
    입력받은 주소를 WGS84 좌표계 좌표로 변환
    """
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
    headers = {'Authorization': f'KakaoAK {KAKAO_REST_API_KEY}'}
 
    response = requests.get(url, headers=headers)
 
    if response.status_code == 200:
        data = response.json()
        if "documents" in data and data["documents"]:
            longitude = float(data["documents"][0]["x"])
            latitude = float(data["documents"][0]["y"])
            return latitude, longitude
        else:
            print(f"No coordinates found for address: {address}")
    else:
        print(f"Error: Status code {response.status_code}")
    
    return None, None

# 데이터프레임의 각 행에 대해 처리
for index, row in charger_df.iterrows():
    if pd.isnull(row['위도']):
        latitude, longitude = convert_address_to_coordinates(row['주소'])
        if latitude is not None and longitude is not None:
            charger_df.at[index, '위도'] = latitude
            charger_df.at[index, '경도'] = longitude

No coordinates found for address: 서울특별시 구로구 디지털로33길 48, 504, B203호
No coordinates found for address: 서울특별시 강동구 강일동 강일리버파크 602동 1303호
No coordinates found for address: 서울특별시 용산구 두핍바위로37길 6-2
No coordinates found for address: 서울특별시 성북구 장위동 231-542 1 층
No coordinates found for address: 서울특별시 성북구 장위동 231-192 1 층
No coordinates found for address: 서울특별시 용산구 티파크1단지) 103-2605
No coordinates found for address: 서울특별시 용산구 티파크1단지) 103동
No coordinates found for address: 서울특별시 동작구 티빌) 102호
No coordinates found for address: 서울특별시 구로구 흥대로161가길 19-19 (구로동) 알토하우스 1층
No coordinates found for address: 서울특별시 강서구 양천로 티엔빌1동
No coordinates found for address: 서울특별시 강동구 티뷰) 길동76/5삼성
No coordinates found for address: 서울특별시 동대문구  동대문구 약령
No coordinates found for address: 서울특별시 동대문구 난계로 234 (신설동) 111
No coordinates found for address: 서울특별시 마포구 큐리티
No coordinates found for address: 서울특별시 금천구  벛꽃로 하이랜드 클래식 816호 영유통
No coordinates found for address: 서울특별시 중구 광화문 내일신문 사옥


In [9]:
#좌표 null drop
charger_df = charger_df.dropna(subset = ['위도'])

#확인
charger_df.isna().sum()

설치년도           0
시도             0
군구            81
주소             0
충전소명           0
시설구분(대)       43
시설구분(소)        0
기종(대)          0
기종(소)          0
운영기관(대)        0
운영기관(소)        0
급속충전량      45017
충전기타입          0
이용자제한          0
충전기ID          0
충전소ID          0
위도경도         721
위도             0
경도             0
dtype: int64

### 전처리 2 : 군구
- 주소의 자치구명을 결측치에 대체

In [10]:
display(charger_df[charger_df['군구'].isna()].head(3))

# 주소에서 자치구명 가져오기
charger_df['군구'] = charger_df['군구'].fillna(charger_df['주소'].str.split()[1][1])

#데이터 확인
charger_df.isna().sum()

,설치년도,시도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),운영기관(대),운영기관(소),급속충전량,충전기타입,이용자제한,충전기ID,충전소ID,위도경도,위도,경도
81653,2023,서울특별시,NaN,서울 강서구 공항대로59길 64,한사랑2차삼성아파트,공동주택시설,아파트,완속,AC완속,타기관(보조금),지에스커넥트,NaN,AC완속,이용자제한,1,GN066096,"37.5546232,126.8641847",37.554623,126.864185
81654,2023,서울특별시,NaN,서울 강서구 공항대로59길 64,한사랑2차삼성아파트,공동주택시설,아파트,완속,AC완속,타기관(보조금),지에스커넥트,NaN,AC완속,이용자제한,2,GN066096,"37.5546232,126.8641847",37.554623,126.864185
81655,2023,서울특별시,NaN,서울 강서구 공항대로59길 64,한사랑2차삼성아파트,공동주택시설,아파트,완속,AC완속,타기관(보조금),지에스커넥트,NaN,AC완속,이용자제한,3,GN066096,"37.5546232,126.8641847",37.554623,126.864185


설치년도           0
시도             0
군구             0
주소             0
충전소명           0
시설구분(대)       43
시설구분(소)        0
기종(대)          0
기종(소)          0
운영기관(대)        0
운영기관(소)        0
급속충전량      45017
충전기타입          0
이용자제한          0
충전기ID          0
충전소ID          0
위도경도         721
위도             0
경도             0
dtype: int64

### 전처리 3 : 시설구분(대)
- 시설구분(대) 결측치는 모두 시설구분(소) : 기타
- 시설구분(대) 결측치 '기타시설'로 채우기

In [11]:
# charger_df[charger_df['시설구분(대)'].isna()]
print('시설구분(대) 고유값 : ', charger_df['시설구분(대)'].unique())

# 시설구분(대)의 결측치는 모두 시설구분(소)가 '기타시설'
charger_df[charger_df['시설구분(소)'] == '기타'][charger_df['시설구분(대)'].isna()]

#시설구분(대) 결측치 '기타시설'로 채우기
charger_df['시설구분(대)'] = charger_df['시설구분(대)'].fillna('기타시설')

#데이터 확인
charger_df.isna().sum()


시설구분(대) 고유값 :  ['주차시설' '기타시설' '관광시설' '상업시설' '공공시설' '근린생활시설' '차량정비시설' '공동주택시설' '교육문화시설'
 '휴게시설' nan]


/var/folders/tz/h0d7hy211kj12my58s3b3m7m0000gn/T/ipykernel_7066/803604735.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  charger_df[charger_df['시설구분(소)'] == '기타'][charger_df['시설구분(대)'].isna()]


설치년도           0
시도             0
군구             0
주소             0
충전소명           0
시설구분(대)        0
시설구분(소)        0
기종(대)          0
기종(소)          0
운영기관(대)        0
운영기관(소)        0
급속충전량      45017
충전기타입          0
이용자제한          0
충전기ID          0
충전소ID          0
위도경도         721
위도             0
경도             0
dtype: int64

### 전처리 4 : 급속충전량, 위도경도
- 급속충전량 결측치 : 45017개. 결측치간 일정한 규칙 찾기 어려워 결측치 처리 하지 않음
- 위도경도 결측치 : 위도, 경도 컬럼 새로 생성하였으니 삭제

In [12]:
#급속충전량 데이터 확인
print('급속충전량 고유값 :' ,  charger_df['급속충전량'].unique())
#위도경도 컬럼 삭제
charger_df = charger_df.drop(columns = '위도경도')
#데이터 확인
charger_df.head(3)

급속충전량 고유값 : ['급속(50kW)' '급속(100kW동시)' '급속(100kW멀티)' '급속(200kW동시)' '급속(400kW동시)'
 '급속(100kW단독)' '초급속(350kW단독)' '완속(30kW단독)' nan]


,설치년도,시도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),운영기관(대),운영기관(소),급속충전량,충전기타입,이용자제한,충전기ID,충전소ID,위도,경도
0,2017,서울특별시,강동구,서울특별시 강동구 고덕로28길 36,암사1동 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,1,ME174003,37.552909,127.137255
1,2017,서울특별시,강동구,서울특별시 강동구 선사로 70,천호유수지 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,1,ME174004,37.544812,127.121840
2,2017,서울특별시,강동구,서울특별시 강동구 진황도로29길 34,천호3동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,1,ME174005,37.538795,127.133399


### 전처리 5 : 좌표로 행정동명 추가
- 카카오API

In [13]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def get_admin_district_name(latitude, longitude, api_key):
    url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'
    headers = {'Authorization': f'KakaoAK {api_key}'}
    response = requests.get(url, headers=headers)
    data = response.json()
    if data.get('documents'):
        region = data['documents'][0]
        region_name = region['region_3depth_name']
        return region_name
    else:
        return None

# 카카오 API 키
api_key = '13ee922882dc101cbce7c7a616661d77'

# 데이터프레임에서 위도와 경도 정보 추출
coordinates = [(row['위도'], row['경도']) for index, row in charger_df.iterrows()]

# 중복된 좌표 제거
unique_coordinates = list(set(coordinates))

# API 요청 결과 캐싱
cached_results = {}
for coord in unique_coordinates:
    cached_results[coord] = None

# 캐싱되지 않은 결과에 대해서만 API 요청
def process_coordinate(coord):
    latitude, longitude = coord
    if cached_results[coord] is None:
        cached_results[coord] = get_admin_district_name(latitude, longitude, api_key)

# 병렬 처리를 사용하여 API 요청 동시에 처리
with ThreadPoolExecutor() as executor:
    executor.map(process_coordinate, unique_coordinates)

# 결과를 DataFrame에 추가
charger_df['행정동명'] = [cached_results[coord] for coord in coordinates]

In [14]:
charger_df.isna().sum()

설치년도           0
시도             0
군구             0
주소             0
충전소명           0
시설구분(대)        0
시설구분(소)        0
기종(대)          0
기종(소)          0
운영기관(대)        0
운영기관(소)        0
급속충전량      45017
충전기타입          0
이용자제한          0
충전기ID          0
충전소ID          0
위도             0
경도             0
행정동명           2
dtype: int64

In [15]:
charger_df

,설치년도,시도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),운영기관(대),운영기관(소),급속충전량,충전기타입,이용자제한,충전기ID,충전소ID,위도,경도,행정동명
0,2017,서울특별시,강동구,서울특별시 강동구 고덕로28길 36,암사1동 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,1,ME174003,37.552909,127.137255,암사동
1,2017,서울특별시,강동구,서울특별시 강동구 선사로 70,천호유수지 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,1,ME174004,37.544812,127.121840,천호동
2,2017,서울특별시,강동구,서울특별시 강동구 진황도로29길 34,천호3동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,1,ME174005,37.538795,127.133399,천호동
3,2017,서울특별시,강서구,서울특별시 강서구 가로공원로 189,가로공원로 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,1,ME174006,37.537263,126.838379,화곡동
4,2017,서울특별시,강서구,서울특별시 강서구 강서로5길 50,곰달래문화복지센터 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,1,ME174007,37.528956,126.849089,화곡동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273225,2023,서울특별시,강동구,서울특별시 영등포구 은행로 25,안원빌딩 지하2층,기타시설,기타,완속,AC완속,타기관(보조금),한화솔루션,NaN,AC완속,이용가능,1,HW000310,37.528696,126.921395,여의도동
273226,2023,서울특별시,강동구,서울특별시 영등포구 은행로 25,안원빌딩 지하2층,기타시설,기타,완속,AC완속,타기관(보조금),한화솔루션,NaN,AC완속,이용가능,2,HW000310,37.528696,126.921395,여의도동
273267,2012,서울특별시,중구,서울특별시 중구 세종대로 110,서울시 신청사,공공시설,공공기관,급속,DC차데모+AC3상,지자체,지자체기타,NaN,DC차데모+AC3상,이용자제한,1,99000001,37.566350,126.978309,태평로1가
273268,2015,서울특별시,양천구,서울특별시 양천구 중앙로 341-3,명화운수,기타시설,기타,급속,AC3상,지자체,지자체기타,NaN,AC3상,이용자제한,1,99000002,37.527824,126.848920,신정동


In [16]:
charger_df.to_csv('전처리_충전소.csv', index = False)